# Computer vision

## Image classification

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nlphuji/flickr30k")
image = dataset['test'][134]["image"] 

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image) 

In [ ]:
from transformers import pipeline

pipe = pipeline("image-classification", "google/mobilenet_v2_1.0_224")
pred = pipe(image) 
print("Predicted class:", pred[0]['label'])

## Object detection

In [ ]:
pipe = pipeline("object-detection", "facebook/detr-resnet-50", revision="no_timm")
outputs = pipe(image, threshold=0.95)

for obj in outputs:
  box = obj['box'] 
  print(f"Detected {obj['label']} with confidence {obj['score']:.2f} at ({box['xmin']}, {box['ymin']}) to ({box['xmax']}, {box['ymax']})")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

ax = plt.gca()
colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k']
plt.imshow(image)  
for n, obj in enumerate(outputs):
  box = obj['box']   
  rect = patches.Rectangle(
    (box['xmin'], box['ymin']),
    box['xmax']-box['xmin'],
    box['ymax']-box['ymin'],
    linewidth=1,
    edgecolor=colors[n],
    facecolor='none')  
  ax.add_patch(rect)  
plt.show() 

## Image background removal

In [ ]:
pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)
outputs = pipe(image)

plt.imshow(outputs)
plt.show()

# Fine-tunning computer vision models

## CV fine-tunning: dataset prep

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ideepankarsharma2003/Midjourney_v6_Classification_small_shuffled")['train']
data_splits = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
from transformers import AutoImageProcessor 

checkpoint = "google/mobilenet_v2_1.0_224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

from torchvision.transforms import Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
transform = Compose([ToTensor(), normalize])

def transforms(examples):
  examples["pixel_values"] = [transform(img.convert("RGB")) for img in examples["image"]]
  del examples["image"]
  return examples

dataset = dataset.with_transform(transforms)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(dataset["train"][0]["pixel_values"].permute(1, 2, 0))
plt.show()

## CV fine-tunning: model classes

In [ ]:
labels = data_splits["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
from transformers import AutoModelForImageClassification 

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

## CV fine-tunning: trainer configuration

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="dataset_finetune",
  learning_rate=6e-5,
  gradient_accumulation_steps=4,
  num_train_epochs=3,
  push_to_hub=False
)

In [ ]:
from transformers import Trainer, DefaultDataCollator

data_collator = DefaultDataCollator()
trainer = Trainer(  
  model=model,
  args=training_args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"],
  processing_class=image_processor,
  data_collator=data_collator
)